# Credit Risk Assessment & Default Prediction

## Overview
This project builds an end-to-end credit risk assessment model to predict the probability
of customer default. It supports data-driven lending decisions and portfolio risk
management for financial institutions by identifying risky customers early and enabling
risk-based strategies.

## Business Objective

The objectives of this project are:
- Identify high-risk customers before credit approval
- Understand key factors influencing default behavior
- Segment customers into actionable risk categories (Low, Medium, High)
- Support informed and responsible lending decisions

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [48]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv(
    '/content/drive/My Drive/credit-risk-assessment/data/raw/credit_data.csv'
)

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


## Data Understanding

The dataset contains customer demographic details, financial attributes,
and repayment history. The target variable indicates whether a customer
defaulted on the next payment cycle.

In [24]:
df.shape

(30000, 25)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [49]:
df.isnull().sum()

,0
ID,0
LIMIT_BAL,0
SEX,0
EDUCATION,0
MARRIAGE,0
AGE,0
PAY_0,0
PAY_2,0
PAY_3,0
PAY_4,0


In [27]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [50]:
df['default.payment.next.month'].value_counts(normalize=True)

,proportion
default.payment.next.month,
0,0.7788
1,0.2212


## Feature Engineering

Non-informative identifiers were removed. Relevant demographic,
financial, and repayment behavior features were selected for modeling.

In [51]:
X = df.drop(columns=['ID', 'default.payment.next.month'])
y = df['default.payment.next.month']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

## Model Building

A Logistic Regression model was developed to estimate the probability
of customer default. Logistic regression is widely used in financial
risk modeling due to its interpretability.

In [53]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

## Model Validation & Evaluation

Model performance was evaluated using classification metrics and ROC-AUC
to assess predictive capability and risk discrimination.

In [55]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_prob))

              precision    recall  f1-score   support

           0       0.82      0.96      0.88      7009
           1       0.65      0.24      0.35      1991

    accuracy                           0.80      9000
   macro avg       0.74      0.60      0.62      9000
weighted avg       0.78      0.80      0.77      9000

ROC-AUC Score: 0.6854859924303394


## Risk Segmentation

Customers were segmented into Low, Medium, and High-risk groups based
on predicted default probabilities to support business decision-making.

In [56]:
risk_df = X_test.copy()
risk_df['default_probability'] = y_prob

risk_df['risk_segment'] = pd.cut(
    risk_df['default_probability'],
    bins=[0, 0.3, 0.6, 1.0],
    labels=['Low Risk', 'Medium Risk', 'High Risk']
)

risk_df['risk_segment'].value_counts()

,count
risk_segment,
Low Risk,7239
Medium Risk,1309
High Risk,452


## Business Insights & Recommendations

- Low Risk customers can be offered standard or preferential credit terms.
- Medium Risk customers require additional verification or reduced credit limits.
- High Risk customers show a high probability of default and should be declined
  or offered collateral-backed products.

This segmentation enables financial institutions to reduce default losses,
optimize portfolio risk, and apply risk-based pricing strategies.

## Key Outcomes

- Developed a logistic regression credit risk model with strong predictive performance
- Optimized classification decisions to minimize financial risk exposure
- Segmented customers into actionable risk categories
- Generated structured outputs for downstream business teams

## Business Impact

The insights from this project help financial institutions to:
- Reduce credit losses
- Apply risk-based pricing
- Improve portfolio risk management
- Make informed, data-driven lending decisions